# 4) Vaja: Dockerizacija Flask s Postgres, Gunicorn in Nginx

This is a step-by-step tutorial that details how to configure Flask to run on Docker with Postgres. For production environments, we'll add on Nginx and Gunicorn. We'll also take a look at how to serve static and user-uploaded media files via Nginx.

## Project Setup

    mkdir services && cd services
    mkdir web && cd web
    mkdir project

Next, let's create a new Flask app.

Add an `__init__.py` file to the "project" directory and configure the first route:

In [ ]:
from flask import Flask, jsonify


app = Flask(__name__)


@app.route("/")
def hello_world():
    return jsonify(hello="world")

Then, to configure the Flask CLI tool to run and manage the app from the command line, add a manage.py file to the "web" directory:

In [ ]:
from flask.cli import FlaskGroup

from project import app


cli = FlaskGroup(app)


if __name__ == "__main__":
    cli()

Create a requirements.txt file in the "web" directory and add Flask as a dependency:

    Flask==1.1.1

Your project structure should look like:

    └── web
        ├── manage.py
        ├── project
        │   └── __init__.py
        └── requirements.txt

## Docker - del01

Install Docker, if you don't already have it, then add a Dockerfile to the "web" directory:

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

# set work directory
WORKDIR /usr/src/app

# set environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1

# install dependencies
RUN pip install --upgrade pip
COPY ./requirements.txt /usr/src/app/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . /usr/src/app/
```

So, we started with a slim-buster-based Docker image for Python 3.8.1. We then set a working directory along with two environment variables:

1. PYTHONDONTWRITEBYTECODE: Prevents Python from writing pyc files to disc (equivalent to python -B option)
2. PYTHONUNBUFFERED: Prevents Python from buffering stdout and stderr (equivalent to python -u option)

Finally, we updated Pip, copied over the requirements.txt file, installed the dependencies, and copied over the Flask app itself.

> Review Docker for Python Developers for more on structuring Dockerfiles as well as some best practices for configuring Docker for Python-based development.

Next, add a docker-compose.yml file to the project root:

```yml
version: '3.7'

services:
  web:
    build: ./services/web
    command: python manage.py run -h 0.0.0.0
    volumes:
      - ./services/web/:/usr/src/app/
    ports:
      - 5000:5000
    env_file:
      - ./.env.dev
```

Then, create a `.env.dev` file in the project root to store environment variables for developmen

    FLASK_APP=project/__init__.py
    FLASK_ENV=development

Build the image:

    docker-compose build

Once the image is built, run the container:

    docker-compose up -d

Navigate to http://localhost:5000/ to again view the hello world sanity check.

> Check for errors in the logs if this doesn't work via docker-compose logs -f.

## Postgres - del02

To configure Postgres, we need to add a new service to the docker-compose.yml file, set up Flask-SQLAlchemy, and install Psycopg2.

First, add a new service called db to docker-compose.yml:

    version: '3.7'

    services:
      web:
        build: ./services/web
        command: python manage.py run -h 0.0.0.0
        volumes:
          - ./services/web/:/usr/src/app/
        ports:
          - 5000:5000
        env_file:
          - ./.env.dev
        depends_on:
          - db
      db:
        image: postgres:12-alpine
        volumes:
          - postgres_data:/var/lib/postgresql/data/
        environment:
          - POSTGRES_USER=hello_flask
          - POSTGRES_PASSWORD=hello_flask
          - POSTGRES_DB=hello_flask_dev

    volumes:
      postgres_data:

To persist the data beyond the life of the container we configured a volume. This config will bind postgres_data to the "/var/lib/postgresql/data/" directory in the container.

We also added an environment key to define a name for the default database and set a username and password.

> Review the "Environment Variables" section of the Postgres Docker Hub page for more info.

Add a DATABASE_URL environment variable to .env.dev as well:

    FLASK_APP=project/__init__.py
    FLASK_ENV=development
    DATABASE_URL=postgresql://hello_flask:hello_flask@db:5432/hello_flask_dev

Then, add a new file called config.py to the "project" directory, where we'll define environment-specific configuration variables:

In [ ]:
import os


basedir = os.path.abspath(os.path.dirname(__file__))


class Config(object):
    SQLALCHEMY_DATABASE_URI = os.getenv("DATABASE_URL", "sqlite://")
    SQLALCHEMY_TRACK_MODIFICATIONS = False

Here, the database is configured based on the DATABASE_URL environment variable that we just defined. Take note of the default value.

Update `__init__.py` to pull in the config on init:

In [ ]:
from flask import Flask, jsonify


app = Flask(__name__)
app.config.from_object("project.config.Config")


@app.route("/")
def hello_world():
    return jsonify(hello="world")

Add Flask-SQLAlchemy and Psycopg2 to requirements.txt:

    Flask==1.1.1
    Flask-SQLAlchemy==2.4.1
    psycopg2-binary==2.8.4

Update `__init__.py` again to create a new SQLAlchemy instance and define a database model:

In [ ]:
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy


app = Flask(__name__)
app.config.from_object("project.config.Config")
db = SQLAlchemy(app)


class User(db.Model):
    __tablename__ = "users"

    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(128), unique=True, nullable=False)
    active = db.Column(db.Boolean(), default=True, nullable=False)

    def __init__(self, email):
        self.email = email


@app.route("/")
def hello_world():
    return jsonify(hello="world")

Finally, update manage.py:

In [ ]:
from flask.cli import FlaskGroup

from project import app, db


cli = FlaskGroup(app)


@cli.command("create_db")
def create_db():
    db.drop_all()
    db.create_all()
    db.session.commit()


if __name__ == "__main__":
    cli()

This registers a new command, create_db, to the CLI so that we can run it from the command line, which we'll use shortly to apply the model to the database.

Build the new image and spin up the two containers:

    docker-compose up -d --build

Create the table:

    docker-compose exec web python manage.py create_db

> Get the following error? `sqlalchemy.exc.OperationalError: (psycopg2.OperationalError) FATAL:  database "hello_flask_dev" does not exist`.  Run docker-compose down -v to remove the volumes along with the containers. Then, re-build the images, run the containers, and apply the migrations.

Ensure the users table was created:

    docker-compose exec db psql --username=hello_flask --dbname=hello_flask_dev
    hello_flask_dev=# \l
    hello_flask_dev=# \c hello_flask_dev
    hello_flask_dev=# \dt
    hello_flask_dev=# \q

You can check that the volume was created as well by running:

    docker volume inspect project_postgres_data

Next, add an entrypoint.sh file to the "web" directory to verify that Postgres is up and healthy before creating the database table and running the Flask development server:

    #!/bin/sh

    if [ "$DATABASE" = "postgres" ]
    then
        echo "Waiting for postgres..."

        while ! nc -z $SQL_HOST $SQL_PORT; do
          sleep 0.1
        done

        echo "PostgreSQL started"
    fi

    python manage.py create_db

    exec "$@"

Take note of the environment variables.

Update the file permissions locally:

    chmod +x services/web/entrypoint.sh

Then, update the Dockerfile to install Netcat, copy over the entrypoint.sh file, and run the file as the Docker entrypoint command:

    # pull official base image
    FROM python:3.8.1-slim-buster

    # set work directory
    WORKDIR /usr/src/app

    # set environment variables
    ENV PYTHONDONTWRITEBYTECODE 1
    ENV PYTHONUNBUFFERED 1

    # install system dependencies
    RUN apt-get update && apt-get install -y netcat

    # install dependencies
    RUN pip install --upgrade pip
    COPY ./requirements.txt /usr/src/app/requirements.txt
    RUN pip install -r requirements.txt

    # copy project
    COPY . /usr/src/app/

    # run entrypoint.sh
    ENTRYPOINT ["/usr/src/app/entrypoint.sh"]

Add the SQL_HOST, SQL_PORT, and DATABASE environment variables, for the entrypoint.sh script, to .env.dev:

    FLASK_APP=project/__init__.py
    FLASK_ENV=development
    DATABASE_URL=postgresql://hello_flask:hello_flask@db:5432/hello_flask_dev
    SQL_HOST=db
    SQL_PORT=5432
    DATABASE=postgres

Test it out again:
- Re-build the images
- Run the containers
- Try http://localhost:5000/

Let's also add a CLI seed command for adding sample users to the users table in manage.py:

In [ ]:
from flask.cli import FlaskGroup

from project import app, db, User


cli = FlaskGroup(app)


@cli.command("create_db")
def create_db():
    db.drop_all()
    db.create_all()
    db.session.commit()


@cli.command("seed_db")
def seed_db():
    db.session.add(User(email="michael@mherman.org"))
    db.session.commit()


if __name__ == "__main__":
    cli()

Try it out:

    docker-compose exec web python manage.py seed_db
    docker-compose exec db psql --username=hello_flask --dbname=hello_flask_dev
    hello_flask_dev=# \c hello_flask_dev
    hello_flask_dev=# select * from users;
    hello_flask_dev=# \q

Despite adding Postgres, we can still create an independent Docker image for Flask by not setting the DATABASE_URL environment variable. To test, build a new image and then run a new container:

    docker build -f ./services/web/Dockerfile -t hello_flask:latest ./services/web
    docker run -p 5001:5000 \
        -e "FLASK_APP=project/__init__.py" -e "FLASK_ENV=development" \
        hello_flask python /usr/src/app/manage.py run -h 0.0.0.0

You should be able to view the hello world sanity check at http://localhost:5001.

## Gunicorn - del03

Moving along, for production environments, let's add Gunicorn, a production-grade WSGI server, to the requirements file:

    Flask==1.1.1
    Flask-SQLAlchemy==2.4.1
    gunicorn==20.0.4
    psycopg2-binary==2.8.4

Since we still want to use Flask's built-in server in development, create a new compose file called docker-compose.prod.yml for production:

Take note of the default command. We're running Gunicorn rather than the Flask development server. We also removed the volume from the web service since we don't need it in production. Finally, we're using separate environment variable files to define environment variables for both services that will be passed to the container at runtime.

.env.prod:
    
    FLASK_APP=project/__init__.py
    FLASK_ENV=production
    DATABASE_URL=postgresql://hello_flask:hello_flask@db:5432/hello_flask_prod
    SQL_HOST=db
    SQL_PORT=5432
    DATABASE=postgres

.env.prod.db:

    POSTGRES_USER=hello_flask
    POSTGRES_PASSWORD=hello_flask
    POSTGRES_DB=hello_flask_prod

Add the two files to the project root. You'll probably want to keep them out of version control, so add them to a .gitignore file.

Bring down the development containers (and the associated volumes with the -v flag):

    docker-compose down -v

Then, build the production images and spin up the containers:

    docker-compose -f docker-compose.prod.yml up -d --build

Verify that the hello_flask_prod database was created along with the users table. Test out http://localhost:5000/.

## Production Dockerfile - del04

Did you notice that we're still running the create_db command, which drops all existing tables and then creates the tables from the models, every time the container is run? This is fine in development, but let's create a new entrypoint file for production.

entrypoint.prod.sh:

    #!/bin/sh

    if [ "$DATABASE" = "postgres" ]
    then
        echo "Waiting for postgres..."

        while ! nc -z $SQL_HOST $SQL_PORT; do
          sleep 0.1
        done

        echo "PostgreSQL started"
    fi

    exec "$@"

Update the file permissions locally:

    chmod +x services/web/entrypoint.prod.sh

To use this file, create a new Dockerfile called Dockerfile.prod for use with production builds:

```Dockerfile
###########
# BUILDER #
###########

# pull official base image
FROM python:3.8.1-slim-buster as builder

# set work directory
WORKDIR /usr/src/app

# set environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1

# install system dependencies
RUN apt-get update && \
    apt-get install -y --no-install-recommends gcc

# lint
RUN pip install --upgrade pip
RUN pip install flake8
COPY . /usr/src/app/
RUN flake8 --ignore=E501,F401 .

# install python dependencies
COPY ./requirements.txt .
RUN pip wheel --no-cache-dir --no-deps --wheel-dir /usr/src/app/wheels -r requirements.txt


#########
# FINAL #
#########

# pull official base image
FROM python:3.8.1-slim-buster

# create directory for the app user
RUN mkdir -p /home/app

# create the app user
RUN addgroup -S app && adduser -S app -G app

# create the appropriate directories
ENV HOME=/home/app
ENV APP_HOME=/home/app/web
RUN mkdir $APP_HOME
WORKDIR $APP_HOME

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
COPY --from=builder /usr/src/app/wheels /wheels
COPY --from=builder /usr/src/app/requirements.txt .
RUN pip install --upgrade pip
RUN pip install --no-cache /wheels/*

# copy entrypoint-prod.sh
COPY ./entrypoint.prod.sh $APP_HOME

# copy project
COPY . $APP_HOME

# chown all the files to the app user
RUN chown -R app:app $APP_HOME

# change to the app user
USER app

# run entrypoint.prod.sh
ENTRYPOINT ["/home/app/web/entrypoint.prod.sh"]
```

Here, we used a Docker multi-stage build to reduce the final image size. Essentially, builder is a temporary image that's used for building the Python wheels. The wheels are then copied over to the final production image and the builder image is discarded.

> You could take the multi-stage build approach a step further and use a single Dockerfile instead of creating two Dockerfiles. Think of the pros and cons of using this approach over two different files.

Did you notice that we created a non-root user? By default, Docker runs container processes as root inside of a container. This is a bad practice since attackers can gain root access to the Docker host if they manage to break out of the container. If you're root in the container, you'll be root on the host.

Update the web service within the docker-compose.prod.yml file to build with Dockerfile.prod:

    web:
      build:
        context: ./services/web
        dockerfile: Dockerfile.prod
      command: gunicorn --bind 0.0.0.0:5000 manage:app
      ports:
        - 5000:5000
      env_file:
        - ./.env.prod
      depends_on:
        - db

Try it out:

    docker-compose -f docker-compose.prod.yml down -v
    docker-compose -f docker-compose.prod.yml up -d --build
    docker-compose -f docker-compose.prod.yml exec web python manage.py create_db

## Nginx - del05